Reading data 

In [104]:
import csv
from collections import defaultdict
import math
# Open the CSV file in read mode
data = []
with open('titanic-homework.csv', 'r') as csvfile:
  csv_reader = csv.reader(csvfile)

  for row in csv_reader:
      data.append(row)

data_length = len(data) -1

In [105]:
#first index is for no, second for yes
def CollectData(data, attr):
    values = defaultdict(lambda: [0, 0])
    for i, row in enumerate(data):
        if i != 0:  # Skip the header row
            if attr == 4:  # Check if the attribute is the fourth one (age)
                age = int(row[attr])
                if 0 <= age <= 20:
                    age_group = 'young'
                elif 20 < age <= 40:
                    age_group = 'middle'
                elif 40 < age <= 100:
                    age_group = 'old'
                else:
                    continue  # Skip invalid age values

                if int(row[-1]) == 1:
                    values[age_group][1] += 1
                else:
                    values[age_group][0] += 1
            else:
                if int(row[-1]) == 1:
                    values[row[attr]][1] += 1
                else:
                    values[row[attr]][0] += 1
    return values

def CalculateEntropyForAttr(data):
    entropies={}
    for key in data.keys():
        first_value = data[key][0]
        second_value = data[key][1]
        sum_values = first_value + second_value

        entropy = 0
        if first_value > 0:
            entropy -= (first_value / sum_values) * math.log2(first_value / sum_values)
        if second_value > 0:
            entropy -= (second_value / sum_values) * math.log2(second_value / sum_values)

        entropies[key] = entropy
    return entropies

def CalculateIntrinsicInfo(attr):
    intrinsic_info= 0
    for value in attr.values():
        temp = (value[0] + value[1]) / data_length
        intrinsic_info -=  temp * math.log2(temp)
    return intrinsic_info

getting occurances of survival for each value for each attribute
Calculating entropies for each value of each argument

In [106]:
decisions = []
for i in range (1, len(data[0])-1):
    #skip the name attr
    if i !=2:
        dict = CollectData(data, i)
        decisions.append(dict)
print(decisions)

entropies = []

for attr in decisions:
    entropies.append(CalculateEntropyForAttr(attr))

print(entropies)

[defaultdict(<function CollectData.<locals>.<lambda> at 0x0000010D87950280>, {'3': [44, 16], '1': [9, 12], '2': [7, 12]}), defaultdict(<function CollectData.<locals>.<lambda> at 0x0000010D87931EE0>, {'male': [53, 7], 'female': [7, 33]}), defaultdict(<function CollectData.<locals>.<lambda> at 0x0000010D87A9D1F0>, {'middle': [31, 19], 'old': [14, 7], 'young': [15, 14]}), defaultdict(<function CollectData.<locals>.<lambda> at 0x0000010D847515E0>, {'1': [15, 12], '0': [34, 25], '3': [4, 2], '4': [2, 1], '2': [3, 0], '5': [2, 0]}), defaultdict(<function CollectData.<locals>.<lambda> at 0x0000010D84751CA0>, {'0': [48, 29], '1': [5, 4], '2': [5, 6], '5': [1, 1], '3': [1, 0]})]
[{'3': 0.8366407419411673, '1': 0.9852281360342516, '2': 0.9494520153879484}, {'male': 0.5197027865043055, 'female': 0.6690158350565576}, {'middle': 0.9580420222262995, 'old': 0.9182958340544896, 'young': 0.9991421039919088}, {'1': 0.9910760598382222, '0': 0.9831491282282878, '3': 0.9182958340544896, '4': 0.918295834054

Calculating conditional entropy for each argument

In [107]:
conditional_entropies= []
for i, attr in enumerate(decisions):
    conditional_entr = 0
    for key in attr.keys():
        conditional_entr += ((decisions[i][key][0] + decisions[i][key][1]) / data_length) * entropies[i][key]
    conditional_entropies.append(conditional_entr)

print(conditional_entropies)



[0.8892782366556035, 0.5794280059252064, 0.9616143464222462, 0.9302951468759139, 0.9543693160498283]


Calculating information gain for each argument

In [108]:
information_gains = []
for value in conditional_entropies:
    information_gains.append (1- value)

print(information_gains)

[0.11072176334439654, 0.4205719940747936, 0.03838565357775381, 0.06970485312408614, 0.04563068395017167]


Calculating gain ratios for each of the attributes

In [109]:
gain_ratios =[]
for i, attr in enumerate(decisions):
    gain_ratios.append(information_gains[i] / CalculateIntrinsicInfo(attr))
print(gain_ratios)

[0.08080530167007237, 0.4331548860228121, 0.025749620775081968, 0.04305209220247883, 0.040288443395889766]
